## Estruturas Criptográficas 2022/23
## TP2. Problema 1
### Grupo 7. Leonardo Berteotti e Paulo R. Pereira


Pretende-se a construção de uma classe Python que implemente um **KEM - *El Gamal***. 

Em primeiro lugar, a classe deve inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits da ordem do grupo cíclico). O **KEM - *El Gamal*** utiliza os parâmetros públicos do protocolo Diffie-Hellman. Por sua vez, as técnicas da família Diffie-Hellman utilizam as propriedades de um grupo cíclico multiplicativo  $\,\mathcal{G}\,\equiv\,\mathbb{Z}^\ast_p\,$  em que  $\,p\,$ é  número primo grande. A ordem deste grupo é $p-1$. Assim, é necessário encontrar um primo $p$ cuja ordem tenha um tamanho igual ou superior ao parâmetro de segurança. Além disso, $p$ tem um divisor primo $q$ grande (de forma a que DLP seja  complexo). Assim,
1. começa-se por gerar o primo $q$ tal que o seu tamanho é igual ou superior a 160 bits (20 bytes);
2. gerar sucessivamente $p_i = q \times 2^i + 1$ até que $p_i - 1$ tenha tamanho igual ou superior ao critério de segurança (passado como parâmetro);
3. é gerado $g\in\mathbb{Z}_p^\ast$ de ordem $q$;
4. obtém-se o tuplo $(p, q, g)$.

Para gerar $g$, uma vez que todo o grupo multiplicativo da forma $\,\mathbb{Z}_p^\ast\,$ é um grupo cíclico de ordem $\,p-1\,$, um gerador deste grupo pode-se determinar por tentativas percorrendo os pequenos primos ($\,2,3,5,\cdots$) e determinando a ordem de cada um. Um primo cuja ordem seja $\,p-1\,$ é um gerador. No entanto, queremos que $g$ tenha ordem $q$. Assim, podemos usar o algoritmo de encontrar geradores de grupos cíclicos, baseado no facto de que se $g$ é um gerador de $\,\mathbb{Z}_p^\ast\,$, então $g^{(p-1)//q}$ é um gerador de ordem q.

O algoritmo consiste em gerar aleatoriamente um elemento g de Zp e calcular h = g^((p-1)/q). Em seguida, verificamos se h^q = 1 e se h é diferente de 1. Se ambas as condições são satisfeitas, então h é um gerador de ordem q.

Deste modo, são implementados os seguintes métodos:
- KeyGen : 
  1. Parâmetros públicos  $\,p,q,g\,$ como no protocolo DH
  2. A chave privada é $\,a\neq 0\in \mathbb{Z}_q\,$ gerada aleatoriamente; 
  3. A chave pública é $\,\beta \equiv  g^a\bmod p$
- KEM $(\beta)$ $\equiv \vartheta \,r \gets \mathbb{Z}_q\!\setminus 0\,\centerdot\, \vartheta \,\mathsf{key}\gets \beta^r\bmod p\,\centerdot\, \vartheta\,\mathsf{enc}\gets g^r\bmod p\,\centerdot\,(\mathsf{key}\,,\,\mathsf{enc})$
- KRev $(a,\mathsf{enc})\;\equiv\;\mathsf{enc}^a\bmod p$

Os métodos DEM e DRev usam a primitiva Authenticated symmetric encryption ChaCha20Poly1305. Neste caso em particular, como a chave simétrica precisa de ter 32 bytes, foi usado um **KDF** para, a partir da chave acordada, gerar a nova chave acordada de 32 bytes. Note-se que para chave ser igual nos dois agentes, os parâmetros que conferem a aleatoriedade devem ser iguais e por isso são passados por argumento.

In [58]:
from sage.all import *
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes

def find_generator(p, q):
    while True:
        g = Zmod(p).random_element()
        h = g**((p-1)//q)
        if h**q == 1 and h != 1:
            return h

def generate_private_key(q):
    # generate a random integer between 1 and q-1 (inclusive)
    a = randrange(1, q)
    # ensure that the element is non-zero
    while a == 0:
        a = randrange(1, q)
    return a

class KEM_ElGamal:
    def __init__(self, s):
        self.s = s

    def key_gen(self):
        q = random_prime(2**160-1,True,2**(160-1))
        i = 0
        while True:
            p = q * 2**i + 1
            if (p-1).bit_length() >= self.s and is_prime(p):
                break
            i += 1
        g = find_generator(p,q)

        # generate_private_key(q) generates a random non-zero element in Zq
        sk = generate_private_key(q)
        pk = power_mod(g, sk, p)
        return p, q, g, sk, pk

    # generate key and key encapsulation
    def KEM(self, pk, p, q, g):
        r = generate_private_key(q)
        key = power_mod(pk, r, p)
        enc = power_mod(g, r, p)
        return key, enc

    # reveals key
    def KRev(self, sk, enc, p):
        return power_mod(enc, sk, p)

    # encapsulates plaintext using the primitive of 
    # authenticated encryption ChaCha20Poly1305.
    def DEM(self, plaintext, key, nonce):
        key_bytes = str(key).encode('utf-8')
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b"salt",
            info=b"additional info",
        )
        cipher_key = hkdf.derive(key_bytes)
        chacha = ChaCha20Poly1305(cipher_key)
        aad = b"authenticated but unencrypted data"
        ciphertext = chacha.encrypt(nonce, plaintext, aad)
        return ciphertext

    def DRev(self, ciphertext, sk, enc, p, nonce):
        key = self.KRev(sk, enc, p)
        key_bytes = str(key).encode('utf-8')
        hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b"salt",
            info=b"additional info",
        )
        cipher_key = hkdf.derive(key_bytes)
        chacha = ChaCha20Poly1305(cipher_key)
        aad = b"authenticated but unencrypted data"
        plaintext = chacha.decrypt(nonce, ciphertext, aad)
        return plaintext

# Test
kem = KEM_ElGamal(1024)
p, q, g, sk, pk = kem.key_gen()

k, e = kem.KEM(pk, p, q, g)

nonce = os.urandom(12)

plaintext = 'hello there :)'
print("Plaintext:\n" + plaintext)
ciphertext = kem.DEM(plaintext.encode('utf-8'), k, nonce)
print("\nCiphertext:")
print(ciphertext.decode('unicode_escape'))
print("\nDecrypted ciphertext:")
decrypted_ciphertext = kem.DRev(ciphertext, sk, e, p, nonce)
print(decrypted_ciphertext.decode('unicode_escape'))



Plaintext:
hello there :)

Ciphertext:
¬hÖÈEÕïRê×2U-únS´ÙEÚÖ-úz

Decrypted ciphertext:
hello there :)


Por fim, pretende-se, a partir do **KEM** já definido, e usando a transformação de Fujisaki-Okamoto, um **PKE** que seja *IND-CCA* seguro.

É então construido um esquema assimétrico $\,E',D'\,$  através de $$E'(x)\;\equiv\;\vartheta\,r \gets h\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\, (e,k) \gets f(y\|r)\,\centerdot\,\vartheta\,c\gets k\oplus r\,\centerdot\,(y, e, c)$$
Portanto, tém-se o seguinte:
1. gerar um *random_generated* $r$ que é resultado do hash a um número pseudo-aleatório;
2. calcular $g(r)$ a partir do novo hash $g$;
3. efetuar o XOR entre o *plaintext* $x$ e o $g(r)$ do ponto 2 de forma a obter $y$;
4. concatenar $y$ com $r$ e obter a chave e o encapsulamento da chave $\,k, e\,$ tal como no método KEM da classe **KEM_ElGamal**;
5. efetuar o XOR da chave $k$ com o $r$ para obter uma ofuscação da chave  $c$.


In [68]:
def xor(a,b):
    return bytes([ x^y for (x,y) in zip(a,b)])

def encryptFOT (pk, plaintext, p, q, g):
    r = hash(ZZ.random_element(0, p-1))
    gr = hash(str(r))
    y = xor(plaintext.encode('utf-8'), gr.to_bytes(len(plaintext.encode('utf-8')), byteorder='little'))
    conc = str(y) + str(r)
    key = power_mod(pk, conc, p)
    enc = power_mod(g, conc, p)
    c = xor(k, r)
    return y, enc, c

plaintext = "secret message"
y, enc, c = encryptFOT(pk, plaintext, p, q, g)

TypeError: unable to convert "b'g\\xe7\\xe8?Wk\\xb71essage'1096679095323017437" to an integer

O algorimo de decifrar será $$D'(y,e,c) \;\equiv\;\vartheta\,k \gets \mathsf{KREv}(e)\,\centerdot\,\vartheta\,r \gets c \oplus k\,\centerdot\,\mathsf{if}\;\;(e,k)\neq f(y\|r) \;\;\mathsf{then}\;\;\bot\;\;\mathsf{else}\;\;y \oplus g(r)$$
Assim, tém-se que:
1. ola
2. x

In [38]:
def decryptFOT (pk, sk, ciphertext, e, c):
    X = "TODO"